Installing tensorflow

In [3]:
%tensorflow_version 1.x # Select module of the tensorflow

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # Select module of the tensorflow`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [4]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-07-29 19:56:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.175.167.123, 54.82.179.106, 3.231.73.243, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.175.167.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.46MB/s    in 2.0s    

2021-07-29 19:56:43 (6.46 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [5]:
#the logs that are created while training 
LOG_DIR = "/root/models/trained"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


In [6]:
!python -c 'import matplotlib as tf; print(tf.__version__)' # Check the version of the tensorflow


3.2.2


In [7]:
%cd 

/root


Cloning Object Detection Models from Github

In [8]:
!git clone https://github.com/tensorflow/models.git # Import required files from the website


Cloning into 'models'...
remote: Enumerating objects: 59469, done.
remote: Counting objects: 100% (948/948), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 59469 (delta 634), reused 868 (delta 579), pack-reused 58521
Receiving objects: 100% (59469/59469), 573.70 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (41261/41261), done.


Testing a File in Cloned Object Detection File 

In [9]:
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
!python setup.py build

/root/models/research
python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [10]:
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'

In [11]:
!pip install tf_slim

     |████████████████████████████████| 352 kB 4.1 MB/s 


In [12]:
!python object_detection/builders/model_builder_test.py

In [13]:
#!python3 --version

!python -c 'import tensorflow as tf; print(tf.__version__)'

1.15.2


In [14]:
!pip show tensorflow

Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.7
Requires: opt-einsum, astor, protobuf, wheel, keras-preprocessing, tensorboard, absl-py, google-pasta, termcolor, wrapt, gast, grpcio, numpy, tensorflow-estimator, six, keras-applications
Required-by: stable-baselines, magenta, kapre


Import from Github

In [15]:
%cd /root/models/
!git clone https://github.com/emurphy7/Odometer_project 

/root/models
Cloning into 'Odometer_project'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 45 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [16]:
import os
import shutil
import glob
import urllib
import tarfile
from requests import get
source = '/root/models/Odometer_project/character'
destination = '/root/models/'
shutil.move('/root/models/Odometer_project/character',  '/root/models/') 


'/root/models/character'

In [17]:
shutil.move('/root/models/character/fine_tuned',  '/root/models/') 

'/root/models/fine_tuned'

Add csv files      
Data Folder : test_labels.csv       
Data Folder : odometer_long.csv


Change name of folder and add files from Desktop

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Converting XML Files to CSV Files

In [20]:
os.getcwd()

'/root/models'

In [21]:
import pandas as pd
import numpy as np

test = pd.read_csv('/root/models/character/data/test_labels.csv')



Downloading Pre-trained Models

In [22]:
%cd ~/models



MODEL = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = 'pretrained_model'

if not (os.path.exists(MODEL_FILE)):
  with open(MODEL_FILE, "wb") as file:
    # get request
    response = get(DOWNLOAD_BASE + MODEL_FILE)
    # write to file
    file.write(response.content)


tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
  shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/root/models


Detection

In [23]:
%cd /root/models/research/
import pathlib
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
# This is needed to display the images.
%matplotlib inline


/root/models/research


In [24]:
# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/root/models/fine_tuned' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/root/models/character/data/object-detection.pbtxt'

NUM_CLASSES = 11


In [25]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


In [26]:
# 
PATH_TO_CKPT = '/root/models/fine_tuned' + '/frozen_inference_graph_v2_300.pb'
detection_graph_v2  =tf.Graph()
with detection_graph_v2.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [27]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [28]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


Commented out section to be run when cropping based off of a new odometer model output 


In [29]:
path = '/root/models/character/images'
os.mkdir(path)
path = '/root/models/character/images/test'
os.mkdir(path)

In [30]:
path = '/root/models/test_cropped'
os.mkdir(path)

In [ ]:
!cp -r /content/drive/MyDrive/ODOMETER_FILTERED_bicubic_result_sharpen/ /root/models/character/images/test

In [ ]:
#make cropped test images
import pandas as pd
test_cropped = pd.read_csv('/root/models/character/data/odometer_long_v2.csv')

In [ ]:
!cd /content/models/character/images/test/ODOMETER_FILTERED_bicubic_result_sharpen
for i in range(len(test_cropped)):
  name = '/root/models/character/images/test/ODOMETER_FILTERED_bicubic_result_sharpen/result_' +  test_cropped['filename'][i]
  im = Image.open(name)
  w, h = im.size
  top = round((test_cropped['box1_xmin'][i]-2)   *( h /100 )) 
  if top < 0:
    top = 0
  left = round((test_cropped['box1_ymin'][i] -2)   * (w / 100))
  if left< 0 :
    left = 0
  bottom = round((test_cropped['box1_ymax'][i]+2)  *( h /100 )) 
  if bottom > h:
    bottom = h 
  right = round((test_cropped['box1_xmax'][i]+2)   *( w /100 ))
  if right > w:
    right = w

 # print(left,right, top, bottom)
  im1 = im.crop((left, top, right, bottom))
  url_result = test_cropped['filename'][i]
  im1.save("/root/models/test_cropped/" + url_result, 'JPEG')
  plt.imshow(im1)



In [27]:
!cp -r  /root/models/test_cropped /content/drive/MyDrive/crop_test_final_sharp_v2

In [ ]:
#!cp -r /content/drive/MyDrive/crop_test_final_sharp /root/models/test_cropped

In [ ]:
cd /root/models/

In [ ]:
#!cp -r /root/models/test_cropped /content/drive/MyDrive/crop_test_final_sharp

In [ ]:
file_names = test_cropped['filename']
# To test the code with your image add the path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/root/models/test_cropped'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, file_names[i]) for i in range(len(test_cropped)) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.1), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


In [ ]:
import pandas as pd
import numpy as np
import math
import networkx as nx
import statistics
columns = ['filename','mileage']
N = range(len(file_names))
df_ = pd.DataFrame(index=N,  columns=columns)
df_ = df_.fillna(0) # with 0s rather than NaNs
file_name = df_['filename']
mileage = df_['mileage']


def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += str(ele)  
    
    # return string  
    return str1 
count = 0;
for image_path in TEST_IMAGE_PATHS:
  file_name[count] = image_path[26:]
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  boxs = output_dict['detection_boxes']
  score = output_dict['detection_scores']
  classification = output_dict['detection_classes']
  #print(classification)
  # Detection V2 
  output_dict_v2 = run_inference_for_single_image(image_np, detection_graph_v2)
  boxs_v2 = output_dict_v2['detection_boxes']
  score_v2 = output_dict_v2['detection_scores']
  classification_v2 = output_dict_v2['detection_classes']
  #vis_util.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    boxs_v2,
  #    classification_v2,
  #    score_v2,
  #    category_index,
  #    instance_masks=output_dict_v2.get('detection_masks'),
  #    use_normalized_coordinates=True,
  #    line_thickness=4, mask_alpha = .1, min_score_thresh = .05, max_boxes_to_draw = 25)
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(image_np)
  #remove characters V1 
  ind = []
  for i in range(output_dict['num_detections']):
    if classification[i] == 11:
      ind.append(i)
  classification = np.delete(classification, ind, axis=0)
  score = np.delete(score, ind, axis=0)
  boxs = np.delete(boxs,ind,axis=0)
  #remove characters V2
  ind = []
  for i in range(output_dict_v2['num_detections']):
    if classification_v2[i] == 11:
      ind.append(i)
  classification_v2 = np.delete(classification_v2, ind, axis=0)
  score_v2 = np.delete(score_v2, ind, axis=0)
  boxs_v2 = np.delete(boxs_v2,ind,axis=0)
  # remove digits under a certain threshold V1
  threshold = .01;
  ind = []
  for i in range(len(classification)):
   if(score[i]) < threshold:
      ind.append(i)
  classification = np.delete(classification, ind, axis=0)
  score = np.delete(score, ind, axis=0)
  boxs = np.delete(boxs,ind,axis=0)

  # remove digits under a certain threshold V2
  threshold = .1;
  ind = []
  for i in range(len(classification_v2)):
   if(score_v2[i]) < threshold:
      ind.append(i)
  classification_v2 = np.delete(classification_v2, ind, axis=0)
  score_v2 = np.delete(score_v2, ind, axis=0)
  boxs_v2 = np.delete(boxs_v2,ind,axis=0)

  # V2 remove boxes in the same area 
  ind = []
  for i in range(len(boxs_v2)):
    for j in range(len(boxs_v2)):
      if (boxs_v2[i][1]> boxs_v2[j][1] and  boxs_v2[i][3] < boxs_v2[j][3]) or abs((boxs_v2[i][1] - boxs_v2[j][1])) < .01 :
        if i != j:
          if (i>j):
            ind.append(i)
          else:
            ind.append(j)
  classification_v2 = np.delete(classification_v2, ind, axis=0)
  score_v2 = np.delete(score_v2, ind, axis=0)
  boxs_v2 = np.delete(boxs_v2,ind,axis=0)
 ## V1 remove boxes in the same area 
  ind = []
  for i in range(len(boxs)):
    for j in range(len(boxs)):
      if (boxs[i][1]> boxs[j][1] and  boxs[i][3] < boxs[j][3]) or abs((boxs[i][1] - boxs[j][1])) < .01 :
        if i != j:
          if (i>j):
            ind.append(i)
          else:
            ind.append(j)
  classification = np.delete(classification, ind, axis=0)
  score = np.delete(score, ind, axis=0)
  boxs = np.delete(boxs,ind,axis=0)


  #Extend V2
  for i in range(len(boxs_v2)):
    width = boxs_v2[i][3] - boxs_v2[i][1]
    if(classification_v2[i] == 2):
      width_add = width
    else:
      width_add = width / 2.5
    boxs_v2[i][3] =boxs_v2[i][3] +width_add
    boxs_v2[i][1] =boxs_v2[i][1]  - width_add

  #Extend V1
  for i in range(len(boxs)):
    width = boxs[i][3] - boxs[i][1]
    height = boxs[i][3] - boxs[i][1]
    if(classification[i] == 2):
      width_add = height / 2.5
    else:
      width_add = width / 2.5
    boxs[i][3] =boxs[i][3] +width_add
    boxs[i][1] =boxs[i][1]  - width_add


  # remove tall / long  boxes V2 
  med_width = []
  med_height = []
  for i in range(len(boxs_v2)):
    med_width.append(abs(boxs_v2[i][3]-boxs_v2[i][1]))
    med_height.append(abs(boxs_v2[i][2]-boxs_v2[i][0]))
  if(len(boxs_v2) > 0):
    med_width = statistics.median(med_width)
    med_height = statistics.median(med_height)
  ind = [] 
  for i in range(len(boxs_v2)):
    if(abs(abs(boxs_v2[i][3]-boxs_v2[i][1]) - med_width) > .15):
      ind.append(i)
    elif (abs(abs(boxs_v2[i][2]-boxs_v2[i][0]) - med_height) > .15):
      ind.append(i)
  classification_v2 = np.delete(classification_v2, ind, axis=0)
  score_v2 = np.delete(score_v2, ind, axis=0)
  boxs_v2 = np.delete(boxs_v2,ind,axis=0)


  # remove tall / long  boxes V1
  med_width = []
  med_height = []
  for i in range(len(boxs)):
    med_width.append(abs(boxs[i][3]-boxs[i][1]))
    med_height.append(abs(boxs[i][2]-boxs[i][0]))
  if(len(boxs) > 0):
    med_width = statistics.median(med_width)
    med_height = statistics.median(med_height)
  ind = [] 
  for i in range(len(boxs)):
    if(abs(abs(boxs[i][3]-boxs[i][1]) - med_width) > .15):
      ind.append(i)
    elif (abs(abs(boxs[i][2]-boxs[i][0]) - med_height) > .15):
      ind.append(i)
  classification = np.delete(classification, ind, axis=0)
  score = np.delete(score, ind, axis=0)
  boxs = np.delete(boxs,ind,axis=0)





  dic = {}
  #############################################################################################
  # check to see if they are close in width and height  ( NEEDS TO BE CHANGED BASED ON MODEL) #
  ############################################################################################# 
  for i in range(len(boxs_v2)):
    width = boxs_v2[i][3] - boxs_v2[i][1]
    for j in range(len(boxs_v2)):
      dif_width = boxs_v2[i][3] - boxs_v2[j][1]
      dif_height = boxs_v2[i][0] - boxs_v2[j][0]
      if (i !=j) and dif_width > 0 and dif_height < .1 and abs(dif_width) < .1 :
        if not(j in dic and dic[j] == i):
          dic[i] = j
  G = nx.DiGraph()
  G.add_edges_from(dic.items())
  try:
    path = nx.dag_longest_path(G)
  except nx.exception.NetworkXUnfeasible: # There's a loop!
    print("The graph has a cycle")
    path = []
  final = []
  for i in range(len(path)):
    final.append(classification_v2[path[i]] -1)
  if len(final) == 0:
    final_int = 0
    print('empty')
  else:
    final_string = listToString(final)
    final_int = int(final_string)
    if(len(final_string) >6 or final_int > 300000 ):
      final_int = int(str(final_int)[:-1])
    print(final_int)
  mileage[count] = final_int;
  # Visualization of the results of a detection.
  count = count + 1;
  #vis_util.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    output_dict['detection_boxes'],
   #   output_dict['detection_classes'],
   #   output_dict['detection_scores'],
   #   category_index,
   #   instance_masks=output_dict.get('detection_masks'),
  #    use_normalized_coordinates=True,
  #    line_thickness=1, mask_alpha = .1, min_score_thresh = .05, max_boxes_to_draw = 25)
  #plt.figure(figsize=IMAGE_SIZE)
  # Visualization of the results of v2 detection
  #plt.imshow(image_np)

  #vis_util.visualize_boxes_and_labels_on_image_array(
  #    image_np,
  #    boxs_v2,
  #    classification_v2,
  #    score_v2,
  #    category_index,
  #    instance_masks=output_dict_v2.get('detection_masks'),
  #    use_normalized_coordinates=True,
  #    line_thickness=4, mask_alpha = .1, min_score_thresh = .01, max_boxes_to_draw = 25)
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(image_np)


In [ ]:
#count = 0 
#for image_path in TEST_IMAGE_PATHS:
#  file_name[count] = image_path[26:]
#  count = count +1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
!cp /root/models/test_cropped /content/drive

In [1]:
compression_opts = dict(method='zip',
                        archive_name='digit_result_v2_extend1_longer_train_2.csv') 
df_.to_csv('digit_result_v2_extend1_longer_train_2.zip', index=False,
          compression=compression_opts) 

NameError: ignored

In [ ]:
!cp -r /root/models/digit_result_v2_extend1_longer_train_2.zip /content/drive/MyDrive

cp: cannot stat '/root/models/digit_result_v1_extend1.zip': No such file or directory
